<span style='font-family:"Times New Roman"'>

## Bibliotecas

Biblotecas necessárias para a análise

In [1]:
# Ambiente
from dotenv import dotenv_values

# Manipulação de dados
import sqlite3
import pandas as pd

# Análise
import phik
from scipy.stats import mstats
from pandas_profiling import ProfileReport


<span style='font-family:"Times New Roman"'>

## Configurações

Configurações de execução

In [2]:
#lear arquivo de configurações
config = dotenv_values("../.env")

# Versão
version = config["VERSION"]
data_new_version = f"data_stats_{version}"
data_old_version = f"data_stats_{version}"

#Leitura de dados
pd.set_option('display.max_columns', 20)



<span style='font-family:"Times New Roman"'>

## Conexões
Banco de dados utilizado

In [3]:
#Conectar com o banco de dados
connection = sqlite3.connect(config["DB_CONFIG"])

<hr style="border:1px solid black"> </hr>
<span style='font-family:"Times New Roman"'>

## Análise Exploratória de Dados (AED)

Essa etapa é fundamental para entendimento das variáveis do banco de dados. Uma vez compreendidos, os dados podem ser utilizados de forma adequada na criação e avaliação de modelos de regressão.

___
<span style='font-family:"Times New Roman"'>
    
## 1. Visulização dos Dados Brutos

Uma gama de informações pode ser extraída dos dados sem uma Análise Exploratória de Dados (AED) formal. A visualização de dados brutos tem como objetivo entendimento (mesmo que mínimo) do problema e avaliação qualidade da informação registrada.


In [4]:
#Ler
df = pd.read_sql_query("SELECT * FROM houses", connection)
df.drop(columns=["index","Id"],inplace=True)
# Amostra aleatória de dados 
print('\033[1m' "\n TABELA 1: AMOSTRA DO BANCO DE DADOS  \n")
df.sample(5,random_state=42)


 TABELA 1: AMOSTRA DO BANCO DE DADOS  



,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
892,20,RL,70.0,8414,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,MnPrv,None,0,2,2006,WD,Normal,154500
1105,60,RL,98.0,12256,Pave,None,IR1,Lvl,AllPub,Corner,...,0,None,None,None,0,4,2010,WD,Normal,325000
413,30,RM,56.0,8960,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,3,2010,WD,Normal,115000
522,50,RM,50.0,5000,Pave,None,Reg,Lvl,AllPub,Corner,...,0,None,None,None,0,10,2006,WD,Normal,159000
1036,20,RL,89.0,12898,Pave,None,IR1,HLS,AllPub,Inside,...,0,None,None,None,0,9,2009,WD,Normal,315500


In [5]:
#Registrar/versionar info dos dados 
try:
    stats = df.describe()
    stats.to_sql(data_new_version, connection)
    
except Exception as e:
    print(e)

Table 'data_stats_V1' already exists.


In [6]:
# Ocorreram alterações nos dados desde a ultima avaliação ?
old_stats = pd.read_sql_query(f"SELECT * FROM {data_old_version}",connection)

if sum(stats[["SalePrice"]].values - old_stats[["SalePrice"]].values)[0] >0:
    print("Sim")
else :
    print("Não")


Não


___
<span style='font-family:"Times New Roman"'>
    
## 2. Perfil do dataset

Uma gama de informações pode ser extraída dos dados sem uma Análise Exploratória de Dados (AED) formal. A visualização de dados brutos tem como objetivo entendimento (mesmo que mínimo) do problema e avaliação qualidade da informação registrada.

**Obs: Os reltórios podem ser melhor observados em HTML (na pasta volume).**


In [7]:
#Salvar perfil de dados
profile = ProfileReport(df, title=f'House Pricing Raw - {version}', explorative=False, minimal=True,vars={"num": {"low_categorical_threshold": 0}} )
profile.to_file(f"../volume/House_Pricing_Raw_{version}.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
print('\033[1m' "\n TABELA 2: PERFIL DOS DADOS  \n")
profile


 TABELA 2: PERFIL DOS DADOS  



___

<span style='font-family:"Times New Roman"'>
    
## 3. Redução de Dimensionalidade do Problema
Primeiro foram removidas as as colunas que continham menos que 50% de preenchimento . Em seguida foi avalida a correlação $\phi(k)$ da variáveis restantes.

In [9]:
#Remover colunas com excesso de dados faltantes
df = df.loc[:, df.isnull().mean() < .5]

#Correlações
print('\033[1m' "\n TABELA 3: CORRELAÇÃO ENTRE AS VARIÁVES \n")
corr=df.phik_matrix()
corr.head()


 TABELA 3: CORRELAÇÃO ENTRE AS VARIÁVES 

interval columns not set, guessing: ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
MSSubClass,1.000000,0.406769,0.444429,0.000000,0.137679,0.300205,0.185140,0.000000,0.102203,0.000000,...,0.266302,0.0,0.066753,0.039937,0.132119,0.034190,0.000000,0.174341,0.264143,0.343144
MSZoning,0.406769,1.000000,0.293982,0.000000,0.204232,0.185537,0.124919,0.000000,0.169312,0.095908,...,0.236584,0.0,0.000000,0.000000,0.000000,0.056552,0.000000,0.256894,0.199592,0.451654
LotFrontage,0.444429,0.293982,1.000000,0.411907,0.106285,0.414648,0.175330,NaN,0.254431,0.181486,...,0.359434,0.0,0.000000,0.496424,0.000000,0.000000,0.028603,0.000000,0.101607,0.330832
LotArea,0.000000,0.000000,0.411907,1.000000,0.271828,0.374979,0.362284,0.000000,0.123182,0.560522,...,0.103734,0.0,0.000000,0.199320,0.000000,0.000000,0.000000,0.000000,0.000000,0.216970
Street,0.137679,0.204232,0.106285,0.271828,1.000000,0.051079,0.171794,0.310971,0.000000,0.106504,...,0.000000,0.0,0.098734,0.000000,0.000000,0.074495,0.020860,0.111295,0.138481,0.000000


A seleção das variáveis mais influentes no preço das casas foi baseada na correlção $\phi(k)$ .A Tabela 4 apresenta a as variáveis com correlação $\phi(k)$ superior a 0.65 com a variável "SalePrice" que foram selecionadas para forma dataset de modelagem. 

In [10]:
print('\033[1m' "\n TABELA 4: VARIÁVEIS COM ALTA CORRELAÇÃO COM O PREÇO DE VENDA  \n")
high_corr = pd.DataFrame(corr["SalePrice"]).query("abs(SalePrice)>0.5")
high_corr.T


 TABELA 4: VARIÁVEIS COM ALTA CORRELAÇÃO COM O PREÇO DE VENDA  



,Neighborhood,OverallQual,YearBuilt,MasVnrArea,ExterQual,BsmtQual,BsmtFinSF1,TotalBsmtSF,HeatingQC,CentralAir,...,2ndFlrSF,GrLivArea,FullBath,KitchenQual,TotRmsAbvGrd,GarageYrBlt,GarageFinish,GarageCars,GarageArea,SalePrice
SalePrice,0.687262,0.820348,0.621852,0.770451,0.675927,0.65562,0.511026,0.576548,0.514728,0.538376,...,0.854449,0.887026,0.613686,0.661979,0.636482,0.532719,0.570523,0.759608,0.700441,1.0


In [11]:
#Reduzir o número de variáveis baseado na correlção com preço
df = df[high_corr.index]

___

<span style='font-family:"Times New Roman"'>
    
## 3. VERSIONAR E SALVAR DADOS
Após a seleção das features , as mesmas são armazedas de acordo com a versão do projeto.

In [12]:
#Separar variáveis categoricas e buméricas
categorical_cols = [cname for cname in df.columns if df[cname].nunique() < 50 and df[cname].dtype == "object"]
numerical_cols = [cname for cname in df.columns if df[cname].dtype in ['int64', 'float64']]

#Salvar features
try:
    df[categorical_cols].to_sql(f"categorical_features_{version}", connection)
    df[numerical_cols ].to_sql( f"numerical_features_{version}", connection)
    
except Exception as e:
    print(e)
 


Table 'categorical_features_V1' already exists.


In [13]:
#Salvar perfil de dados
profile = ProfileReport(df, title=f'House Pricing Features - {version}', explorative=True, minimal=False,vars={"num": {"low_categorical_threshold": 0}} )
profile.to_file(f"../volume/House_Pricing_Features_{version}.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]